<a href="https://colab.research.google.com/github/etuckerman/CNN-Image-Classification-Architecture/blob/main/CNN_Image_Classification_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN architecture
def create_cnn_model(input_shape, num_classes):
    """
    Creates a basic CNN model for image classification.

    Args:
    input_shape (tuple): The shape of the input images (height, width, channels).
    num_classes (int): The number of classes in the classification problem.

    Returns:
    model (tf.keras.Model): The compiled CNN model.
    """

    # Create a sequential model
    model = Sequential()

    # Convolutional layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    # Max-pooling layer 1
    model.add(MaxPooling2D((2, 2)))

    # Convolutional layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # Max-pooling layer 2
    model.add(MaxPooling2D((2, 2)))

    # Convolutional layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    # Max-pooling layer 3
    model.add(MaxPooling2D((2, 2)))

    # Flatten the output
    model.add(Flatten())

    # Fully connected layer 1
    model.add(Dense(128, activation='relu'))

    # Output layer with SoftMax activation
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
input_shape = (224, 224, 3)  # Adjust according to your image size and channels
num_classes = 10  # Adjust according to your classification problem

model = create_cnn_model(input_shape, num_classes)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0